In [57]:
import os
import re
import pandas as pd
import numpy as np

In [58]:
df = pd.read_parquet('test.parquet')
df

,ORGAO_LICITANTE,NUM_LICITACAO,OBJ_LICITACAO,MODALIDADE_LICITACAO,PROCESSO,DATA_ABERTURA,VALOR_ESTIMADO,CODIGO_SISTEMA_COMPRAS,SISTEMA_COMPRAS,NOME_RESPONSAVEL,treated_text,IOB
0,None,024/2002,aquisi- ção ção de conectores,Tomada de Preços de Material,None,10/02/2003,None,None,None,None,AVISO DE LICITAÇÃO A COMPANHIA ENERGÉTICA DE B...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,None,005/2002,Aquisição de materiais permanentes para uso na...,TOMADA DE PREÇOS,053.000.918/2002,11/10/2002,None,None,None,None,AVISO DE LICITAÇÃO PROCESSO N.º 053.000.918/20...,O O O O O O B-PROCESSO I-PROCESSO O O B-MODALI...
2,None,04/2018,Execucao de 05 (Cinco) Edificios habitacionais...,CONCORRENCIA,392.003.366/2017,15 de marco de 2018,None,None,None,CLAYTON FERREIRA ARAGAO,AVISO DE LICITACAO CONCORRENCIA No 04/2018 Obj...,O O O B-MODALIDADE_LICITACAO O B-NUM_LICITACAO...
3,None,240/2019,Registro de precos para aquisicao de pecas e a...,aquisicao,092.005543/2019,23/10/2019,None,974200,www.comprasnet.gov.br,NARA MAGALHAES DA SILVA,AVISO DE LICITACAO PREGAO ELETRONICO PE No 240...,O O O O O O O B-NUM_LICITACAO I-NUM_LICITACAO ...
4,None,241/2019,Registro de precos para aquisicao de materiais...,aquisicao,092.004513/2019,23/10/2019,None,974200,www. c o m p r a s n e t . g o v. b r,SILVIO SANTOS GONCALVES SOARES,AVISO DE LICITACAO PREGAO ELETRONICO PE No 241...,O O O O O O O B-NUM_LICITACAO I-NUM_LICITACAO ...
...,...,...,...,...,...,...,...,...,...,...,...,...
126,None,224/2014,"aquisição de materiais de copa, cozinha, limpe...",PREGÃO ELETRÔNICO,414.000.216/2014,None,"116.236,56",None,http://www.compras.df.gov.br,None,AVISO DE LICITAÇÃO NOVA DATA PREGÃO ELETRÔNICO...,O O O O O B-MODALIDADE_LICITACAO I-MODALIDADE_...
127,ACOMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO...,144/2012,Registro de Preços para fornecimento de reagen...,PREGÃO ELETRÔNICO,092.006540/2012,16 de outubro de 2012,"204.314,58",None,www.licitacoes-e.com.br,None,PREGÃO ELETRÔNICO Nº 144/2012 ACOMPANHIA DE SA...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...
128,COMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO ...,05/2007,aquisição de peças/acessórios rios para equipa...,TOMADA DE PREÇOS,00092.000.301/2007,09 de maio de 2007,None,None,None,None,AVISO DE LICITAÇÕES TOMADA DE PREÇOS Nº 05/200...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
129,Centrais de Abastecimento do Distrito Federal S.A,05/2018,contratação de - empresa para a prestação de s...,PREGÃO ELETRÔNICO,071.000040/2018,None,"75.000,00",926245,www.comprasnet.gov.br,None,AVISO DE LICITAÇÃO EXCLUSIVA PARA ME E EPP PRE...,O O O O O O O O B-MODALIDADE_LICITACAO I-MODAL...


In [59]:
df.isnull().sum()

ORGAO_LICITANTE            82
NUM_LICITACAO               2
OBJ_LICITACAO               1
MODALIDADE_LICITACAO        1
PROCESSO                   15
DATA_ABERTURA              19
VALOR_ESTIMADO             50
CODIGO_SISTEMA_COMPRAS     71
SISTEMA_COMPRAS            31
NOME_RESPONSAVEL          111
treated_text                0
IOB                         0
dtype: int64

In [60]:
ORGAO_LICITANTE = []
VALOR_ESTIMADO = []
NOME_RESPONSAVEL = []

for i in range(len(df)):
    if df['ORGAO_LICITANTE'][i] != None:
        ORGAO_LICITANTE.append(i)
    if df['VALOR_ESTIMADO'][i] != None:
        VALOR_ESTIMADO.append(i)
    if df['NOME_RESPONSAVEL'][i] != None:
        NOME_RESPONSAVEL.append(i)

In [61]:
len(ORGAO_LICITANTE), len(VALOR_ESTIMADO), len(NOME_RESPONSAVEL)

(49, 81, 20)

In [62]:
NOME_RESPONSAVEL_1 = NOME_RESPONSAVEL[:(len(NOME_RESPONSAVEL_set)*50)//100]
NOME_RESPONSAVEL_2 = NOME_RESPONSAVEL[(len(NOME_RESPONSAVEL)*50)//100:]

ORGAO_LICITANTE_set = set(ORGAO_LICITANTE).difference(set(NOME_RESPONSAVEL_2))
ORGAO_LICITANTE_1 = list(ORGAO_LICITANTE_set)[:(len(ORGAO_LICITANTE_set)*50)//100]
ORGAO_LICITANTE_2 = set(list(ORGAO_LICITANTE_set)[(len(ORGAO_LICITANTE_set)*50)//100:]).difference(set(NOME_RESPONSAVEL_1))

VALOR_ESTIMADO_set = set(VALOR_ESTIMADO).difference(set(ORGAO_LICITANTE_2))
VALOR_ESTIMADO_1 = list(VALOR_ESTIMADO_set)[:(len(VALOR_ESTIMADO_set)*50)//100]
VALOR_ESTIMADO_2 = set(list(VALOR_ESTIMADO_set)[(len(VALOR_ESTIMADO_set)*50)//100:]).difference(set(NOME_RESPONSAVEL_1))
VALOR_ESTIMADO_2 = set(list(VALOR_ESTIMADO_set)[(len(VALOR_ESTIMADO_set)*50)//100:]).difference(set(ORGAO_LICITANTE_1))

In [63]:
len(ORGAO_LICITANTE_1), len(ORGAO_LICITANTE_2), len(VALOR_ESTIMADO_1), len(VALOR_ESTIMADO_2), len(NOME_RESPONSAVEL_1), len(NOME_RESPONSAVEL_2)

(21, 21, 36, 35, 10, 10)

In [64]:
lic = []

lic.extend(NOME_RESPONSAVEL_1)
lic.extend(ORGAO_LICITANTE_1)
lic.extend(VALOR_ESTIMADO_1)

lic = sorted(list(set(lic)))

not_lic = []

not_lic.extend(NOME_RESPONSAVEL_2)
not_lic.extend(ORGAO_LICITANTE_2)
not_lic.extend(VALOR_ESTIMADO_2)

not_lic = sorted(list(set(not_lic)))

In [65]:
len(lic), len(not_lic)

(57, 66)

In [66]:
for i in lic:
    for j in not_lic:
        if i == j:
            not_lic.pop(not_lic.index(j))

for i in lic:
    for j in not_lic:
        if i == j:
            print(i)

In [67]:
df_lic = df.iloc[lic]
df_not_lic = df.iloc[not_lic]

In [68]:
len(df_lic), len(df_not_lic)

(57, 57)

In [69]:
df_real = df.drop(df_lic.index)
df_real = df_real.drop(df_not_lic.index)

In [70]:
len(df_real)

17

In [71]:
testa = pd.concat([df_lic, df_real[:int(len(df)*0.5) - len(df_lic)]])
testb = pd.concat([df_real[int(len(df)*0.5) - len(df_lic):], df_not_lic])

In [72]:
len(testa), len(testb), len(df)

(65, 66, 131)

In [73]:
i = 0
for col in list(df.columns):
    print(f'{col}: {((len(testa) - list(testa.isnull().sum())[i]) * 100) / (len(df) - list(df.isnull().sum())[i])}%')
    i += 1

ORGAO_LICITANTE: 55.10204081632653%
NUM_LICITACAO: 48.83720930232558%
OBJ_LICITACAO: 49.23076923076923%
MODALIDADE_LICITACAO: 49.23076923076923%
PROCESSO: 50.0%
DATA_ABERTURA: 50.0%
VALOR_ESTIMADO: 45.67901234567901%
CODIGO_SISTEMA_COMPRAS: 51.666666666666664%
SISTEMA_COMPRAS: 49.0%
NOME_RESPONSAVEL: 95.0%
treated_text: 49.61832061068702%
IOB: 49.61832061068702%


In [74]:
i = 0
for col in list(df.columns):
    print(f'{col}: {((len(testb) - list(testb.isnull().sum())[i]) * 100) / (len(df) - list(df.isnull().sum())[i])}%')
    i += 1

ORGAO_LICITANTE: 44.89795918367347%
NUM_LICITACAO: 51.16279069767442%
OBJ_LICITACAO: 50.76923076923077%
MODALIDADE_LICITACAO: 50.76923076923077%
PROCESSO: 50.0%
DATA_ABERTURA: 50.0%
VALOR_ESTIMADO: 54.32098765432099%
CODIGO_SISTEMA_COMPRAS: 48.333333333333336%
SISTEMA_COMPRAS: 51.0%
NOME_RESPONSAVEL: 5.0%
treated_text: 50.38167938931298%
IOB: 50.38167938931298%


In [75]:
testa = testa.reset_index(drop=True)
testb = testb.reset_index(drop=True)

In [76]:
testa.to_parquet('testa.parquet')
testb.to_parquet('testb.parquet')

In [77]:
testa = pd.read_parquet('testa.parquet')
testb = pd.read_parquet('testb.parquet')

len(testa), len(testb)

(65, 66)